<a href="https://colab.research.google.com/gist/priyanshusharma16/88f8691efa8ad5c31ae141e539862bdb/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.callbacks import ReduceLROnPlateau

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Load data
data = pd.read_json('/content/Sarcasm_Headlines_Dataset.json', lines=True)

In [ ]:
data.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28619 entries, 0 to 28618
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   is_sarcastic  28619 non-null  int64 
 1   headline      28619 non-null  object
 2   article_link  28619 non-null  object
dtypes: int64(1), object(2)
memory usage: 670.9+ KB


In [ ]:
data.shape

(28619, 3)

In [ ]:
#checking for null values in data
data.isnull().sum()

is_sarcastic    0
headline        0
article_link    0
dtype: int64

In [ ]:
data.is_sarcastic.value_counts()

is_sarcastic
0    14985
1    13634
Name: count, dtype: int64

In [ ]:
#checking for duplicate values
data['headline'].duplicated().sum()

116

In [ ]:
# Drop duplicate headlines
data = data.drop(data[data['headline'].duplicated()].index, axis=0)

In [ ]:
#rechecking for duplicate values
data['headline'].duplicated().sum()

0

In [ ]:
# Drop unnecessary columns
data = data.drop(columns=['article_link'])

In [ ]:
data.head()

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...


In [ ]:
# Preprocessing functions
stop_words = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop_words.update(punctuation)

lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer("english")

def clean_text(text):
    words = text.split()
    words = [word.lower() for word in words]
    words = [re.sub(r'[^\w\s]', '', word) for word in words]
    words = [word for word in words if word.isalpha()]
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    words = [stemmer.stem(word) for word in words]
    return ' '.join(words)

data['clean_headline'] = data['headline'].apply(clean_text)

In [ ]:
data.head()

,is_sarcastic,headline,clean_headline
0,1,thirtysomething scientists unveil doomsday clo...,thirtysometh scientist unveil doomsday clock h...
1,0,dem rep. totally nails why congress is falling...,dem rep total nail congress fall short gender ...
2,0,eat your veggies: 9 deliciously different recipes,eat veggi delici differ recip
3,1,inclement weather prevents liar from getting t...,inclement weather prevent liar get work
4,1,mother comes pretty close to using word 'strea...,mother come pretti close use word stream correct


In [ ]:
# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(
    data['clean_headline'], data['is_sarcastic'], test_size=0.20, random_state=42
)

In [ ]:
# Label encoding the target variable
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

print("Label Encoding - Sample labels:")
print("Original:", train_labels[:5].values)
print("Encoded:", train_labels_encoded[:5])

Label Encoding - Sample labels:
Original: [1 0 0 1 0]
Encoded: [1 0 0 1 0]


In [ ]:
# Tokenization and One-Hot Encoding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data)
vocab_size = len(tokenizer.word_index) + 1

train_sequences = tokenizer.texts_to_sequences(train_data)
test_sequences = tokenizer.texts_to_sequences(test_data)

max_length = max([len(seq) for seq in train_sequences])
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

print("One-Hot Encoding - Sample padded sequence:")
print(train_padded[0])

One-Hot Encoding - Sample padded sequence:
[   2  178 1320 3511 3240 4349 1657 1214  554  701   37  116    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [ ]:
import os

# Download GloVe embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

# Load the GloVe embeddings
def load_glove_embeddings(glove_path):
    embeddings_index = {}
    with open(glove_path, 'r', encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

glove_path = 'glove.6B.100d.txt'
embeddings_index = load_glove_embeddings(glove_path)

# Create an embedding matrix
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print('Embedding matrix shape:', embedding_matrix.shape)

--2024-06-20 18:35:06--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-06-20 18:35:06--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-06-20 18:35:06--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
from tensorflow.keras.layers import Bidirectional

# Building the model using pretrained GloVe embeddings
model_glove = Sequential()
model_glove.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model_glove.add(SpatialDropout1D(0.2))
model_glove.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
model_glove.add(Dense(1, activation='sigmoid'))

model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)

history_glove = model_glove.fit(train_padded, train_labels_encoded, epochs=10, batch_size=64,
                                validation_data=(test_padded, test_labels_encoded), callbacks=[reduce_lr])

# Evaluate the model
accuracy_glove = model_glove.evaluate(test_padded, test_labels_encoded)
print("GloVe Embedding Model Accuracy:", accuracy_glove[1])

Epoch 1/10
357/357 [==============================] - 318s 866ms/step - loss: 0.6136 - accuracy: 0.6592 - val_loss: 0.5633 - val_accuracy: 0.7090 - lr: 0.0010
Epoch 2/10
357/357 [==============================] - 292s 818ms/step - loss: 0.5684 - accuracy: 0.7016 - val_loss: 0.5357 - val_accuracy: 0.7344 - lr: 0.0010
Epoch 3/10
357/357 [==============================] - 288s 806ms/step - loss: 0.5357 - accuracy: 0.7270 - val_loss: 0.5093 - val_accuracy: 0.7443 - lr: 0.0010
Epoch 4/10
357/357 [==============================] - 295s 827ms/step - loss: 0.5107 - accuracy: 0.7442 - val_loss: 0.4928 - val_accuracy: 0.7551 - lr: 0.0010
Epoch 5/10
357/357 [==============================] - 303s 848ms/step - loss: 0.4916 - accuracy: 0.7601 - val_loss: 0.4817 - val_accuracy: 0.7636 - lr: 0.0010
Epoch 6/10
357/357 [==============================] - 298s 834ms/step - loss: 0.4738 - accuracy: 0.7716 - val_loss: 0.4727 - val_accuracy: 0.7700 - lr: 0.0010
Epoch 7/10
357/357 [==========================

In [ ]:
from tensorflow.keras.optimizers import Adam

# Building the model with hyperparameter tuning
model_tuned = Sequential()
model_tuned.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False))
model_tuned.add(SpatialDropout1D(0.3))
model_tuned.add(Bidirectional(LSTM(150, dropout=0.3, recurrent_dropout=0.3)))
model_tuned.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=0.001)
model_tuned.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

history_tuned = model_tuned.fit(train_padded, train_labels_encoded, epochs=15, batch_size=32,
                                validation_data=(test_padded, test_labels_encoded), callbacks=[reduce_lr])

# Evaluate the model
accuracy_tuned = model_tuned.evaluate(test_padded, test_labels_encoded)
print("Tuned Model Accuracy:", accuracy_tuned[1])

Epoch 1/15
713/713 [==============================] - 738s 1s/step - loss: 0.6308 - accuracy: 0.6423 - val_loss: 0.5800 - val_accuracy: 0.6985 - lr: 0.0010
Epoch 2/15
713/713 [==============================] - 748s 1s/step - loss: 0.5832 - accuracy: 0.6864 - val_loss: 0.5354 - val_accuracy: 0.7264 - lr: 0.0010
Epoch 3/15
713/713 [==============================] - 769s 1s/step - loss: 0.5519 - accuracy: 0.7135 - val_loss: 0.5108 - val_accuracy: 0.7439 - lr: 0.0010
Epoch 4/15
713/713 [==============================] - 751s 1s/step - loss: 0.5271 - accuracy: 0.7323 - val_loss: 0.4942 - val_accuracy: 0.7564 - lr: 0.0010
Epoch 5/15
713/713 [==============================] - 746s 1s/step - loss: 0.5099 - accuracy: 0.7417 - val_loss: 0.4840 - val_accuracy: 0.7636 - lr: 0.0010
Epoch 6/15
713/713 [==============================] - 748s 1s/step - loss: 0.4971 - accuracy: 0.7516 - val_loss: 0.4779 - val_accuracy: 0.7667 - lr: 0.0010
Epoch 7/15
713/713 [==============================] - 764s 1s/st